In [38]:
#Testing functionality with Franke function,these tests are done without any noise 
#added to the response variable

import numpy as np
import math
from sklearn.preprocessing import PolynomialFeatures, StandardScaler
from random import random, seed
from sklearn.metrics import mean_squared_error,r2_score,mean_absolute_error,mean_squared_log_error
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler,PolynomialFeatures

from sklearn.pipeline import Pipeline
import pandas as pd
from imageio import imread
from sklearn.linear_model import LinearRegression, Ridge, Lasso
from sklearn.utils import shuffle
np.random.seed(244)


x = np.arange(0, 1, 0.001)
y = np.arange(0, 1, 0.001)




#print(x)
def FrankeFunction(x,y):
    term1 = 0.75*np.exp(-(0.25*(9*x-2)**2) - 0.25*((9*y-2)**2))
    term2 = 0.75*np.exp(-((9*x+1)**2)/49.0 - 0.1*(9*y+1))
    term3 = 0.5*np.exp(-(9*x-7)**2/4.0 - 0.25*((9*y-3)**2))
    term4 = -0.2*np.exp(-(9*x-4)**2 - (9*y-7)**2)
    return term1 + term2 + term3 + term4


z = FrankeFunction(x, y)

X_train,X_test,y_train,y_test=train_test_split(np.column_stack((x, y)),z,test_size=0.2)

y_scale=np.mean(y_train)
# Bootstrap and KFold class definitions
class BootstrapKfold:
    def __init__(self):
        pass

    def bootstrap(self, n, function, degree, z, X_train, X_test, y_train, y_test, alpha):
        """
        Perform bootstrap resampling and calculate Cross-Validated Mean Squared Errors (MSE) for regression models.

        Parameters:
            n (int): Number of bootstrap samples.
            function (function): Regression function to be evaluated.
            degree (int): Polynomial degree for regression.
            z (array): Target values for regression.
            X_train (array): Training features.
            X_test (array): Testing features.
            y_train (array): Training target values.
            y_test (array): Testing target values.
            alpha (float): Regularization parameter (alpha) for Ridge and Lasso regression.

        Returns:
            float: Cross-validated mean training MSE.
            float: Cross-validated mean test MSE.
        """
        CV_test_MSE = 0
        CV_train_MSE = 0

        for i in range(n):
            indices = np.random.choice(X_train.shape[0], size=z.shape[0], replace=True)
            X_boot = X_train[indices]
            y_boot = y_train[indices]

            train_MSE, test_MSE = function(degree, X_boot, X_test, y_boot, y_test, alpha)

            CV_test_MSE += test_MSE
            CV_train_MSE += train_MSE

        CV_test_MSE /= n
        CV_train_MSE /= n

        return CV_train_MSE, CV_test_MSE

    def kfold(self, K, function, degree, z, X,alpha):
        """
        Perform K-Fold cross-validation and calculate Mean Squared Errors (MSE) for regression models.

        Parameters:
            K (int): Number of folds.
            function (function): Regression function to be evaluated.
            degree (int): Polynomial degree for regression.
            z (array): Target values for regression.
            X_train (array): Training features.
            X_test (array): Testing features.
            y_train (array): Training target values.
            y_test (array): Testing target values.
            alpha (float): Regularization parameter (alpha) for Ridge and Lasso regression.

        Returns:
            float: Cross-validated mean training MSE.
            float: Cross-validated mean test MSE.
        """
        X=shuffle(X, random_state=0)
        z=shuffle(z, random_state=0)
        fold_size = int(math.ceil(len(z) / K))
        CV_test_MSE = 0
        CV_train_MSE = 0

        for i in range(len(z)):
            test_start = i * fold_size
            test_end = min((i + 1) * fold_size, len(z))
            start_stop = np.arange(test_start, test_end, 1)
            X_test_fold = X[start_stop]
            y_test_fold = z[start_stop]
            rest = np.setdiff1d(np.arange(z.shape[0]), start_stop)
            X_train_fold = X[rest]
            y_train_fold = z[rest]

            train_MSE, test_MSE = function(degree, X_train_fold, X_test_fold, y_train_fold, y_test_fold, alpha)

            CV_test_MSE += test_MSE
            CV_train_MSE += train_MSE

            if test_end == len(z):
                break

        CV_test_MSE /= K
        CV_train_MSE /= K

        return CV_train_MSE, CV_test_MSE

# RegressionClass definition
class RegressionClass:
    def __init__(self):
        pass

    def ols(self, degree, X_train, X_test, y_train, y_test, alpha):
        """
        Perform Ordinary Least Squares regression.

        Parameters:
            degree (int): Polynomial degree for regression.
            X_train (array): Training features.
            X_test (array): Testing features.
            y_train (array): Training target values.
            y_test (array): Testing target values.
            alpha (float): Regularization parameter (not used in OLS).

        Returns:
            float: Training MSE.
            float: Test MSE.
        """
        
        #create design matrices for test and training data
        pipeline = Pipeline([('poly', PolynomialFeatures(degree=degree, interaction_only=False, include_bias=False))
                            ])
        
        
        X_train = pipeline.fit_transform(X_train)
        X_test = pipeline.fit_transform(X_test)
        
        
        #Center the design matrices column wise with X_trains column means
        scaler =StandardScaler(with_std=False)
        scaler.fit(X_train)
        X_train=scaler.transform(X_train)
        X_test=scaler.transform(X_test)
        
        #Center y_train
        y_train = y_train- y_scale
        #compute optimal parameters the minimizes the least squares
        fit = np.linalg.pinv(X_train.T @ X_train) @ (X_train).T @ y_train
        
        #fit the paramters to training and test design matrices
        y_hat_train = X_train @ fit.T
        
        #add intercept to get prediction on uncentered data
        y_hat_test = X_test @ fit.T+ y_scale
        #Compute the mean squares error
        mse_train_test = np.array([mean_squared_error(y_train, y_hat_train), mean_squared_error(y_test, y_hat_test)])
        return mse_train_test

    def ridge(self, degree, X_train, X_test, y_train, y_test, alpha):
        """
        Perform Ridge regression.

        Parameters:
            degree (int): Polynomial degree for regression.
            X_train (array): Training features.
            X_test (array): Testing features.
            y_train (array): Training target values.
            y_test (array): Testing target values.
            alpha (float): Regularization parameter for Ridge regression.

        Returns:
            float: Training MSE.
            float: Test MSE.
        """
        pipeline = Pipeline([('poly', PolynomialFeatures(degree=degree, interaction_only=False, include_bias=False)) 
                             ])
        X_train = pipeline.fit_transform(X_train)
        X_test = pipeline.fit_transform(X_test)
        scaler =StandardScaler(with_std=False)
        scaler.fit(X_train)
        X_train=scaler.transform(X_train)
        X_test=scaler.transform(X_test)
        W = X_train.T @ X_train
        y_train = y_train- y_scale
        fit = np.linalg.pinv(X_train.T @ X_train + alpha * np.eye(W.shape[1])) @ (X_train).T @ y_train
        y_hat_train = X_train @ fit.T
        y_hat_test = X_test @ fit.T+ y_scale
        mse_train_test = np.array([mean_squared_error(y_train, y_hat_train), mean_squared_error(y_test, y_hat_test)])
        return mse_train_test

    def lasso(self, degree, X_train, X_test, y_train, y_test, alpha):
        """
        Perform Lasso regression.

        Parameters:
            degree (int): Polynomial degree for regression.
            X_train (array): Training features.
            X_test (array): Testing features.
            y_train (array): Training target values.
            y_test (array): Testing target values.
            alpha (float): Regularization parameter for Lasso regression.

        Returns:
            float: Training MSE.
            float: Test MSE.
        """
        pipeline = Pipeline([('poly', PolynomialFeatures(degree=degree, interaction_only=False, include_bias=False)),
                             ])
        X_train = pipeline.fit_transform(X_train)
        X_test = pipeline.fit_transform(X_test)
        
        scaler =StandardScaler(with_std=False)
        scaler.fit(X_train)
        X_train=scaler.transform(X_train)
        X_test=scaler.transform(X_test)
        model = Lasso(alpha=alpha,tol=1, max_iter=1000)
        #y_train = y_train- y_scale
        model.fit(X_train, y_train)
        
        y_hat_train = model.predict(X_train)
        y_hat_test = model.predict(X_test)
        mse_train_test = np.array([mean_squared_error(y_train, y_hat_train), mean_squared_error(y_test, y_hat_test)])
        return mse_train_test

# RunRegression class definition
class RunRegression(RegressionClass, BootstrapKfold):
    def __init__(self):
        self.regression_functions = {
            'ols': self.ols,
            'ridge': self.ridge,
            'lasso': self.lasso
        }

    def run(self, regression_type, alpha_values, degrees, X_train, X_test, y_train, y_test, use_bootstrap=False,
            use_kfold=False):
        """
        Run regression models with specified parameters.

        Parameters:
            regression_type (str): Type of regression ('ols', 'ridge', or 'lasso').
            alpha_values (list): List of alpha values for Ridge and Lasso regression.
            degrees (list): List of polynomial degrees for regression models.
            X_train (array): Training features.
            X_test (array): Testing features.
            y_train (array): Training target values.
            y_test (array): Testing target values.
            use_bootstrap (bool): Flag to indicate if bootstrap resampling should be performed.
            use_kfold (bool): Flag to indicate if K-Fold cross-validation should be performed.

        Returns:
            pandas.DataFrame: Dataframe containing regression results.
        """
        results = []

        for alpha in alpha_values:
            alpha_results = []

            for degree in degrees:
                mse_train_test = self.regression_functions[regression_type](degree, X_train, X_test, y_train, y_test,
                                                                            alpha)
                alpha_result = {
                    'alpha': alpha,
                    'degree': degree,
                    'mse_train': mse_train_test[0],
                    'mse_test': mse_train_test[1],
                }

                if use_bootstrap:
                    cv_train_mse, cv_test_mse = self.bootstrap(5, self.regression_functions[regression_type],
                                                               degree, z, X_train, X_test, y_train, y_test, alpha)
                    alpha_result['cv_train_mse'] = cv_train_mse
                    alpha_result['cv_test_mse'] = cv_test_mse

                if use_kfold:
                    cv_train_mse, cv_test_mse = self.kfold(5, self.regression_functions[regression_type],
                                                           degree, z,np.column_stack((x, y)) , alpha)
                    alpha_result['cv_train_mse'] = cv_train_mse
                    alpha_result['cv_test_mse'] = cv_test_mse

                alpha_results.append(alpha_result)

            results.extend(alpha_results)

        result_df = pd.DataFrame(results)
        return result_df
    

# Create an instance of the RunRegression class
regression_instance = RegressionClass()
run_regression_instance = RunRegression()

# Define the alpha values and degrees
alpha_values = [0.00001, 0.00002, 0.00004, 0.0005]
degrees = list(range(1,6))

# Specify the type of regression ('ols', 'ridge', or 'lasso')
regression_type = 'ridge'  # Change to 'lasso' or 'ols' to run other regressions

# Call the run method to perform the specified regression
result_df_ridge = run_regression_instance.run(regression_type, alpha_values,degrees,X_train, X_test, y_train, y_test,use_bootstrap=True, use_kfold=False)


result_df_ridge

regression_type = 'ols'  # OLS regression

# Set alpha to 0 when calling the run method for OLS
result_df_ols = run_regression_instance.run(regression_type, [0], degrees,  X_train, X_test, y_train, y_test,use_bootstrap=True, use_kfold=False)

result_df_ols


regression_type = 'lasso'  # Lasso regression

result_df_lasso = run_regression_instance.run(regression_type,alpha_values, degrees, X_train, X_test, y_train, y_test,use_bootstrap=True, use_kfold=False)

result_df_lasso


from IPython.display import display

# Assuming you have multiple DataFrames df1, df2, df3
display(result_df_ridge)
display(result_df_ols)
display(result_df_lasso)

,alpha,degree,mse_train,mse_test,cv_train_mse,cv_test_mse
0,0.00001,1,0.029768,0.032526,0.030965,0.032937
1,0.00001,2,0.029131,0.032569,0.029253,0.032782
2,0.00001,3,0.013352,0.012933,0.013544,0.013366
3,0.00001,4,0.003864,0.003837,0.004370,0.004316
4,0.00001,5,0.003758,0.003833,0.003824,0.003945
5,0.00002,1,0.029768,0.032526,0.029637,0.032568
6,0.00002,2,0.029131,0.032569,0.028770,0.032774
7,0.00002,3,0.013352,0.012933,0.013240,0.013100
8,0.00002,4,0.003864,0.003838,0.004071,0.004077
9,0.00002,5,0.003759,0.003828,0.003691,0.003957


,alpha,degree,mse_train,mse_test,cv_train_mse,cv_test_mse
0,0,1,0.029768,0.032526,0.029356,0.032638
1,0,2,0.029131,0.032569,0.029234,0.032690
2,0,3,0.013352,0.012933,0.013125,0.013062
3,0,4,0.003864,0.003837,0.004370,0.004349
4,0,5,0.003757,0.003839,0.003805,0.004038


,alpha,degree,mse_train,mse_test,cv_train_mse,cv_test_mse
0,0.00001,1,0.029768,0.032527,0.030258,0.032428
1,0.00001,2,0.029657,0.032578,0.029877,0.032744
2,0.00001,3,0.029138,0.032381,0.028878,0.032376
3,0.00001,4,0.028601,0.032078,0.027873,0.032335
4,0.00001,5,0.028181,0.031796,0.027077,0.031713
5,0.00002,1,0.029768,0.032527,0.029911,0.032550
6,0.00002,2,0.029658,0.032577,0.030278,0.032656
7,0.00002,3,0.029138,0.032380,0.029508,0.032511
8,0.00002,4,0.028600,0.032077,0.029492,0.032084
9,0.00002,5,0.028180,0.031794,0.029273,0.031740


In [45]:
#Testing the code on the Franke Function,now using skitlearn model.fit() and model.predict() to see if the on gets the same
#minimization paramters i.e yeilding the same result as the code above.
import numpy as np
import math
from sklearn.preprocessing import PolynomialFeatures, StandardScaler
from random import random, seed
from sklearn.metrics import mean_squared_error,r2_score,mean_absolute_error,mean_squared_log_error
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler,PolynomialFeatures

from sklearn.pipeline import Pipeline
import pandas as pd
from imageio import imread
from sklearn.linear_model import LinearRegression, Ridge, Lasso
from sklearn.utils import shuffle
np.random.seed(244)


x = np.arange(0, 1, 0.001)
y = np.arange(0, 1, 0.001)



#print(x)
def FrankeFunction(x,y):
    term1 = 0.75*np.exp(-(0.25*(9*x-2)**2) - 0.25*((9*y-2)**2))
    term2 = 0.75*np.exp(-((9*x+1)**2)/49.0 - 0.1*(9*y+1))
    term3 = 0.5*np.exp(-(9*x-7)**2/4.0 - 0.25*((9*y-3)**2))
    term4 = -0.2*np.exp(-(9*x-4)**2 - (9*y-7)**2)
    return term1 + term2 + term3 + term4


z = FrankeFunction(x, y)

X_train,X_test,y_train,y_test=train_test_split(np.column_stack((x, y)),z,test_size=0.2)

y_scale=np.mean(y_train)
# Bootstrap and KFold class definitions
class BootstrapKfold:
    def __init__(self):
        pass

    def bootstrap(self, n, function, degree, z, X_train, X_test, y_train, y_test, alpha):
        """
        Perform bootstrap resampling and calculate Cross-Validated Mean Squared Errors (MSE) for regression models.

        Parameters:
            n (int): Number of bootstrap samples.
            function (function): Regression function to be evaluated.
            degree (int): Polynomial degree for regression.
            z (array): Target values for regression.
            X_train (array): Training features.
            X_test (array): Testing features.
            y_train (array): Training target values.
            y_test (array): Testing target values.
            alpha (float): Regularization parameter (alpha) for Ridge and Lasso regression.

        Returns:
            float: Cross-validated mean training MSE.
            float: Cross-validated mean test MSE.
        """
        CV_test_MSE = 0
        CV_train_MSE = 0

        for i in range(n):
            indices = np.random.choice(X_train.shape[0], size=z.shape[0], replace=True)
            X_boot = X_train[indices]
            y_boot = y_train[indices]

            train_MSE, test_MSE = function(degree, X_boot, X_test, y_boot, y_test, alpha)

            CV_test_MSE += test_MSE
            CV_train_MSE += train_MSE

        CV_test_MSE /= n
        CV_train_MSE /= n

        return CV_train_MSE, CV_test_MSE

    def kfold(self, K, function, degree, z, X, alpha):
        """
        Perform K-Fold cross-validation and calculate Mean Squared Errors (MSE) for regression models.

        Parameters:
            K (int): Number of folds.
            function (function): Regression function to be evaluated.
            degree (int): Polynomial degree for regression.
            z (array): Target values for regression.
            X_train (array): Training features.
            X_test (array): Testing features.
            y_train (array): Training target values.
            y_test (array): Testing target values.
            alpha (float): Regularization parameter (alpha) for Ridge and Lasso regression.

        Returns:
            float: Cross-validated mean training MSE.
            float: Cross-validated mean test MSE.
        """
        X=shuffle(X, random_state=0)
        z=shuffle(z, random_state=0)
        fold_size = int(math.ceil(len(z) / K))
        CV_test_MSE = 0
        CV_train_MSE = 0

        for i in range(len(z)):
            test_start = i * fold_size
            test_end = min((i + 1) * fold_size, len(z))
            start_stop = np.arange(test_start, test_end, 1)
            X_test_fold = X[start_stop]
            y_test_fold = z[start_stop]
            rest = np.setdiff1d(np.arange(z.shape[0]), start_stop)
            X_train_fold = X[rest]
            y_train_fold = z[rest]

            train_MSE, test_MSE = function(degree, X_train_fold, X_test_fold, y_train_fold, y_test_fold, alpha)

            CV_test_MSE += test_MSE
            CV_train_MSE += train_MSE

            if test_end == len(z):
                break

        CV_test_MSE /= K
        CV_train_MSE /= K

        return CV_train_MSE, CV_test_MSE

# RegressionClass definition
class RegressionClass:
    def __init__(self):
        pass

    def ols(self, degree, X_train, X_test, y_train, y_test, alpha):
        """
        Perform Ordinary Least Squares regression.

        Parameters:
            degree (int): Polynomial degree for regression.
            X_train (array): Training features.
            X_test (array): Testing features.
            y_train (array): Training target values.
            y_test (array): Testing target values.
            alpha (float): Regularization parameter (not used in OLS).

        Returns:
            float: Training MSE.
            float: Test MSE.
        """
        pipeline = Pipeline([('poly', PolynomialFeatures(degree=degree , interaction_only=False, include_bias=False))
                             ])
        X_train = pipeline.fit_transform(X_train)
        X_test = pipeline.fit_transform(X_test)
        
        scaler =StandardScaler(with_std=False)
        scaler.fit(X_train)
        X_train=scaler.transform(X_train)
        X_test=scaler.transform(X_test)
        model = LinearRegression()
        
        model.fit(X_train, y_train)
        
        y_hat_train = model.predict(X_train)
        y_hat_test = model.predict(X_test)
        mse_train_test = np.array([mean_squared_error(y_train, y_hat_train), mean_squared_error(y_test, y_hat_test)])
        return mse_train_test

    def ridge(self, degree, X_train, X_test, y_train, y_test, alpha):
        """
        Perform Ridge regression.

        Parameters:
            degree (int): Polynomial degree for regression.
            X_train (array): Training features.
            X_test (array): Testing features.
            y_train (array): Training target values.
            y_test (array): Testing target values.
            alpha (float): Regularization parameter for Ridge regression.

        Returns:
            float: Training MSE.
            float: Test MSE.
        """
        pipeline = Pipeline([('poly', PolynomialFeatures(degree=degree , interaction_only=False, include_bias=False))
                            ])
        X_train = pipeline.fit_transform(X_train)
        X_test = pipeline.fit_transform(X_test)
        
        scaler =StandardScaler(with_std=False)
        scaler.fit(X_train)
        X_train=scaler.transform(X_train)
        X_test=scaler.transform(X_test)
        
        model = Ridge(alpha=alpha)
        model.fit(X_train, y_train)
        
        y_hat_train = model.predict(X_train)
        y_hat_test = model.predict(X_test)
        mse_train_test = np.array([mean_squared_error(y_train, y_hat_train), mean_squared_error(y_test, y_hat_test)])
        return mse_train_test


    def lasso(self, degree, X_train, X_test, y_train, y_test, alpha):
        """
        Perform Lasso regression.

        Parameters:
            degree (int): Polynomial degree for regression.
            X_train (array): Training features.
            X_test (array): Testing features.
            y_train (array): Training target values.
            y_test (array): Testing target values.
            alpha (float): Regularization parameter for Lasso regression.

        Returns:
            float: Training MSE.
            float: Test MSE.
        """
        pipeline = Pipeline([('poly', PolynomialFeatures(degree=degree, interaction_only=False, include_bias=False)),
                             ])
        X_train = pipeline.fit_transform(X_train)
        X_test = pipeline.fit_transform(X_test)
        
        scaler =StandardScaler(with_std=False)
        scaler.fit(X_train)
        X_train=scaler.transform(X_train)
        X_test=scaler.transform(X_test)
        model = Lasso(alpha=alpha,tol=1, max_iter=1000)
       
        model.fit(X_train, y_train)
        
        y_hat_train = model.predict(X_train)
        y_hat_test = model.predict(X_test)
        mse_train_test = np.array([mean_squared_error(y_train, y_hat_train), mean_squared_error(y_test, y_hat_test)])
        return mse_train_test

# RunRegression class definition
class RunRegression(RegressionClass, BootstrapKfold):
    def __init__(self):
        self.regression_functions = {
            'ols': self.ols,
            'ridge': self.ridge,
            'lasso': self.lasso
        }

    def run(self, regression_type, alpha_values, degrees, X_train, X_test, y_train, y_test, use_bootstrap=False,
            use_kfold=False):
        """
        Run regression models with specified parameters.

        Parameters:
            regression_type (str): Type of regression ('ols', 'ridge', or 'lasso').
            alpha_values (list): List of alpha values for Ridge and Lasso regression.
            degrees (list): List of polynomial degrees for regression models.
            X_train (array): Training features.
            X_test (array): Testing features.
            y_train (array): Training target values.
            y_test (array): Testing target values.
            use_bootstrap (bool): Flag to indicate if bootstrap resampling should be performed.
            use_kfold (bool): Flag to indicate if K-Fold cross-validation should be performed.

        Returns:
            pandas.DataFrame: Dataframe containing regression results.
        """
        results = []

        for alpha in alpha_values:
            alpha_results = []

            for degree in degrees:
                mse_train_test = self.regression_functions[regression_type](degree, X_train, X_test, y_train, y_test,
                                                                            alpha)
                alpha_result = {
                    'alpha': alpha,
                    'degree': degree,
                    'mse_train': mse_train_test[0],
                    'mse_test': mse_train_test[1],
                }

                if use_bootstrap:
                    cv_train_mse, cv_test_mse = self.bootstrap(5, self.regression_functions[regression_type],
                                                               degree, z, X_train, X_test, y_train, y_test, alpha)
                    alpha_result['cv_train_mse'] = cv_train_mse
                    alpha_result['cv_test_mse'] = cv_test_mse

                if use_kfold:
                    cv_train_mse, cv_test_mse = self.kfold(5, self.regression_functions[regression_type],
                                                           degree, z,np.column_stack((x, y)) , alpha)
                    alpha_result['cv_train_mse'] = cv_train_mse
                    alpha_result['cv_test_mse'] = cv_test_mse

                alpha_results.append(alpha_result)

            results.extend(alpha_results)

        result_df = pd.DataFrame(results)
        return result_df
    

# Create an instance of the RunRegression class
regression_instance = RegressionClass()
run_regression_instance = RunRegression()

# Define the alpha values and degrees
alpha_values = [0.001, 0.01, 0.1, 1]
degrees = list(range(1,6))

# Specify the type of regression ('ols', 'ridge', or 'lasso')
regression_type = 'ridge'  # Change to 'lasso' or 'ols' to run other regressions

# Call the run method to perform the specified regression
result_df_ridge = run_regression_instance.run(regression_type, alpha_values,degrees,X_train, X_test, y_train, y_test,use_bootstrap=True, use_kfold=True)


result_df_ridge

regression_type = 'ols'  # OLS regression

# Set alpha to 0 when calling the run method for OLS
result_df_ols = run_regression_instance.run(regression_type, [0], degrees,  X_train, X_test, y_train, y_test,use_bootstrap=True, use_kfold=False)

result_df_ols


regression_type = 'lasso'  # Lasso regression

result_df_lasso = run_regression_instance.run(regression_type,alpha_values, degrees, X_train, X_test, y_train, y_test,use_bootstrap=True, use_kfold=False)

result_df_lasso


from IPython.display import display

# Assuming you have multiple DataFrames df1, df2, df3
display(result_df_ridge)
display(result_df_ols)
display(result_df_lasso)

,alpha,degree,mse_train,mse_test,cv_train_mse,cv_test_mse
0,0.001,1,0.029768,0.032526,0.030304,0.030394
1,0.001,2,0.029131,0.032568,0.029786,0.029978
2,0.001,3,0.013353,0.012950,0.013246,0.013334
3,0.001,4,0.003978,0.004001,0.003967,0.004011
4,0.001,5,0.004221,0.004121,0.004200,0.004248
5,0.010,1,0.029768,0.032526,0.030304,0.030394
6,0.010,2,0.029131,0.032565,0.029786,0.029978
7,0.010,3,0.013373,0.013116,0.013268,0.013355
8,0.010,4,0.007292,0.007479,0.007307,0.007360
9,0.010,5,0.005205,0.005222,0.005168,0.005210


,alpha,degree,mse_train,mse_test,cv_train_mse,cv_test_mse
0,0,1,0.029768,0.032526,0.029224,0.032566
1,0,2,0.029131,0.032569,0.029135,0.032586
2,0,3,0.013352,0.012933,0.013048,0.013001
3,0,4,0.003864,0.003837,0.003868,0.003844
4,0,5,0.003757,0.003839,0.003657,0.003951


,alpha,degree,mse_train,mse_test,cv_train_mse,cv_test_mse
0,0.001,1,0.029780,0.032587,0.030270,0.032460
1,0.001,2,0.029780,0.032587,0.029974,0.032765
2,0.001,3,0.029208,0.032335,0.028954,0.032337
3,0.001,4,0.028619,0.031984,0.027894,0.032236
4,0.001,5,0.028170,0.031675,0.027047,0.031563
5,0.010,1,0.030966,0.034204,0.031111,0.034252
6,0.010,2,0.030966,0.034204,0.031559,0.034324
7,0.010,3,0.030966,0.034204,0.031264,0.034497
8,0.010,4,0.030966,0.034204,0.031787,0.034267
9,0.010,5,0.030966,0.034204,0.032051,0.034092


In [28]:
#Here we add noise to the response,and see what happens to the prediction.It seams like when i run it for deiffernt noise,
# and number of points,sometimes one can see 
#underfitting here when adding noise therefore I tried with polynomial complexity up to 29 for this test. 

import numpy as np 
import math
from sklearn.preprocessing import PolynomialFeatures, StandardScaler
from random import random, seed
from sklearn.metrics import mean_squared_error,r2_score,mean_absolute_error,mean_squared_log_error
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler,PolynomialFeatures

from sklearn.pipeline import Pipeline
import pandas as pd
from imageio import imread
from sklearn.linear_model import LinearRegression, Ridge, Lasso
from sklearn.utils import shuffle
#np.random.seed(244)


x = np.arange(0, 1, 0.01)
y = np.arange(0, 1, 0.01)




#print(x)
def FrankeFunction(x,y):
    term1 = 0.75*np.exp(-(0.25*(9*x-2)**2) - 0.25*((9*y-2)**2))
    term2 = 0.75*np.exp(-((9*x+1)**2)/49.0 - 0.1*(9*y+1))
    term3 = 0.5*np.exp(-(9*x-7)**2/4.0 - 0.25*((9*y-3)**2))
    term4 = -0.2*np.exp(-(9*x-4)**2 - (9*y-7)**2)
    return term1 + term2 + term3 + term4


z = FrankeFunction(x, y)+2*np.random.rand(100)

X_train,X_test,y_train,y_test=train_test_split(np.column_stack((x, y)),z,test_size=0.2)

y_scale=np.mean(y_train)
# Bootstrap and KFold class definitions
class BootstrapKfold:
    def __init__(self):
        pass

    def bootstrap(self, n, function, degree, z, X_train, X_test, y_train, y_test, alpha):
        """
        Perform bootstrap resampling and calculate Cross-Validated Mean Squared Errors (MSE) for regression models.

        Parameters:
            n (int): Number of bootstrap samples.
            function (function): Regression function to be evaluated.
            degree (int): Polynomial degree for regression.
            z (array): Target values for regression.
            X_train (array): Training features.
            X_test (array): Testing features.
            y_train (array): Training target values.
            y_test (array): Testing target values.
            alpha (float): Regularization parameter (alpha) for Ridge and Lasso regression.

        Returns:
            float: Cross-validated mean training MSE.
            float: Cross-validated mean test MSE.
        """
        CV_test_MSE = 0
        CV_train_MSE = 0

        for i in range(n):
            indices = np.random.choice(X_train.shape[0], size=z.shape[0], replace=True)
            X_boot = X_train[indices]
            y_boot = y_train[indices]

            train_MSE, test_MSE = function(degree, X_boot, X_test, y_boot, y_test, alpha)

            CV_test_MSE += test_MSE
            CV_train_MSE += train_MSE

        CV_test_MSE /= n
        CV_train_MSE /= n

        return CV_train_MSE, CV_test_MSE

    def kfold(self, K, function, degree, z, X,alpha):
        """
        Perform K-Fold cross-validation and calculate Mean Squared Errors (MSE) for regression models.

        Parameters:
            K (int): Number of folds.
            function (function): Regression function to be evaluated.
            degree (int): Polynomial degree for regression.
            z (array): Target values for regression.
            X_train (array): Training features.
            X_test (array): Testing features.
            y_train (array): Training target values.
            y_test (array): Testing target values.
            alpha (float): Regularization parameter (alpha) for Ridge and Lasso regression.

        Returns:
            float: Cross-validated mean training MSE.
            float: Cross-validated mean test MSE.
        """
        X=shuffle(X, random_state=0)
        z=shuffle(z, random_state=0)
        fold_size = int(math.ceil(len(z) / K))
        CV_test_MSE = 0
        CV_train_MSE = 0

        for i in range(len(z)):
            test_start = i * fold_size
            test_end = min((i + 1) * fold_size, len(z))
            start_stop = np.arange(test_start, test_end, 1)
            X_test_fold = X[start_stop]
            y_test_fold = z[start_stop]
            rest = np.setdiff1d(np.arange(z.shape[0]), start_stop)
            X_train_fold = X[rest]
            y_train_fold = z[rest]

            train_MSE, test_MSE = function(degree, X_train_fold, X_test_fold, y_train_fold, y_test_fold, alpha)

            CV_test_MSE += test_MSE
            CV_train_MSE += train_MSE

            if test_end == len(z):
                break

        CV_test_MSE /= K
        CV_train_MSE /= K

        return CV_train_MSE, CV_test_MSE

# RegressionClass definition
class RegressionClass:
    def __init__(self):
        pass
    def ols(self, degree, X_train, X_test, y_train, y_test, alpha):
        """
        Perform Ordinary Least Squares regression.

        Parameters:
            degree (int): Polynomial degree for regression.
            X_train (array): Training features.
            X_test (array): Testing features.
            y_train (array): Training target values.
            y_test (array): Testing target values.
            alpha (float): Regularization parameter (not used in OLS).

        Returns:
            float: Training MSE.
            float: Test MSE.
        """
        
        #create design matrices for test and training data
        pipeline = Pipeline([('poly', PolynomialFeatures(degree=degree, interaction_only=False, include_bias=False))
                            ])
        
        
        X_train = pipeline.fit_transform(X_train)
        X_test = pipeline.transform(X_test)
        
        
        #Center the design matrices column wise with X_trains column means
        scaler =StandardScaler()
        scaler.fit(X_train)
        X_train=scaler.transform(X_train)
        X_test=scaler.transform(X_test)
        
        #Center y_train
        y_train = y_train- y_scale
        #compute optimal parameters the minimizes the least squares
        fit = np.linalg.pinv(X_train.T @ X_train) @ (X_train).T @ y_train
        
        #fit the paramters to training and test design matrices
        y_hat_train = X_train @ fit.T
        
        #add intercept to get prediction on uncentered data
        y_hat_test = X_test @ fit.T+ y_scale
        #Compute the mean squares error
        mse_train_test = np.array([mean_squared_error(y_train, y_hat_train), mean_squared_error(y_test, y_hat_test)])
        return mse_train_test

    def ridge(self, degree, X_train, X_test, y_train, y_test, alpha):
        """
        Perform Ridge regression.

        Parameters:
            degree (int): Polynomial degree for regression.
            X_train (array): Training features.
            X_test (array): Testing features.
            y_train (array): Training target values.
            y_test (array): Testing target values.
            alpha (float): Regularization parameter for Ridge regression.

        Returns:
            float: Training MSE.
            float: Test MSE.
        """
        pipeline = Pipeline([('poly', PolynomialFeatures(degree=degree, interaction_only=False, include_bias=False)) 
                             ])
        X_train = pipeline.fit_transform(X_train)
        X_test = pipeline.transform(X_test)
        scaler =StandardScaler()
        scaler.fit(X_train)
        X_train=scaler.transform(X_train)
        X_test=scaler.transform(X_test)
        W = X_train.T @ X_train
        y_train = y_train- y_scale
        fit = np.linalg.pinv(X_train.T @ X_train + alpha * np.eye(W.shape[1])) @ (X_train).T @ y_train
        y_hat_train = X_train @ fit.T
        y_hat_test = X_test @ fit.T+ y_scale
        mse_train_test = np.array([mean_squared_error(y_train, y_hat_train), mean_squared_error(y_test, y_hat_test)])
        return mse_train_test

    def lasso(self, degree, X_train, X_test, y_train, y_test, alpha):
        """
        Perform Lasso regression.

        Parameters:
            degree (int): Polynomial degree for regression.
            X_train (array): Training features.
            X_test (array): Testing features.
            y_train (array): Training target values.
            y_test (array): Testing target values.
            alpha (float): Regularization parameter for Lasso regression.

        Returns:
            float: Training MSE.
            float: Test MSE.
        """
        pipeline = Pipeline([('poly', PolynomialFeatures(degree=degree, interaction_only=False, include_bias=False)),
                             ])
        X_train = pipeline.fit_transform(X_train)
        X_test = pipeline.fit_transform(X_test)
        
        scaler =StandardScaler(with_std=False)
        scaler.fit(X_train)
        X_train=scaler.transform(X_train)
        X_test=scaler.transform(X_test)
        model = Lasso(alpha=alpha,tol=1, max_iter=1000)
        #y_train = y_train- y_scale
        model.fit(X_train, y_train)
        
        y_hat_train = model.predict(X_train)
        y_hat_test = model.predict(X_test)
        mse_train_test = np.array([mean_squared_error(y_train, y_hat_train), mean_squared_error(y_test, y_hat_test)])
        return mse_train_test

# RunRegression class definition
class RunRegression(RegressionClass, BootstrapKfold):
    def __init__(self):
        self.regression_functions = {
            'ols': self.ols,
            'ridge': self.ridge,
            'lasso': self.lasso
        }

    def run(self, regression_type, alpha_values, degrees, X_train, X_test, y_train, y_test, use_bootstrap=False,
            use_kfold=False):
        """
        Run regression models with specified parameters.

        Parameters:
            regression_type (str): Type of regression ('ols', 'ridge', or 'lasso').
            alpha_values (list): List of alpha values for Ridge and Lasso regression.
            degrees (list): List of polynomial degrees for regression models.
            X_train (array): Training features.
            X_test (array): Testing features.
            y_train (array): Training target values.
            y_test (array): Testing target values.
            use_bootstrap (bool): Flag to indicate if bootstrap resampling should be performed.
            use_kfold (bool): Flag to indicate if K-Fold cross-validation should be performed.

        Returns:
            pandas.DataFrame: Dataframe containing regression results.
        """
        results = []

        for alpha in alpha_values:
            alpha_results = []

            for degree in degrees:
                mse_train_test = self.regression_functions[regression_type](degree, X_train, X_test, y_train, y_test,
                                                                            alpha)
                alpha_result = {
                    'alpha': alpha,
                    'degree': degree,
                    'mse_train': mse_train_test[0],
                    'mse_test': mse_train_test[1],
                }

                if use_bootstrap:
                    cv_train_mse, cv_test_mse = self.bootstrap(5, self.regression_functions[regression_type],
                                                               degree, z, X_train, X_test, y_train, y_test, alpha)
                    alpha_result['cv_train_mse'] = cv_train_mse
                    alpha_result['cv_test_mse'] = cv_test_mse

                if use_kfold:
                    cv_train_mse, cv_test_mse = self.kfold(5, self.regression_functions[regression_type],
                                                           degree, z,np.column_stack((x, y)) , alpha)
                    alpha_result['cv_train_mse'] = cv_train_mse
                    alpha_result['cv_test_mse'] = cv_test_mse

                alpha_results.append(alpha_result)

            results.extend(alpha_results)

        result_df = pd.DataFrame(results)
        return result_df
    

# Create an instance of the RunRegression class
regression_instance = RegressionClass()
run_regression_instance = RunRegression()

# Define the alpha values and degrees
alpha_values = [0.00001, 0.00002, 0.00004, 0.0005]
degrees = list(range(1,30))

# Specify the type of regression ('ols', 'ridge', or 'lasso')
regression_type = 'ridge'  # Change to 'lasso' or 'ols' to run other regressions

# Call the run method to perform the specified regression
result_df_ridge = run_regression_instance.run(regression_type, alpha_values,degrees,X_train, X_test, y_train, y_test,use_bootstrap=True, use_kfold=False)


result_df_ridge

regression_type = 'ols'  # OLS regression

# Set alpha to 0 when calling the run method for OLS
result_df_ols = run_regression_instance.run(regression_type, [0], degrees,  X_train, X_test, y_train, y_test,use_bootstrap=True, use_kfold=False)

result_df_ols


regression_type = 'lasso'  # Lasso regression

result_df_lasso = run_regression_instance.run(regression_type,alpha_values, degrees, X_train, X_test, y_train, y_test,use_bootstrap=True, use_kfold=False)

result_df_lasso


from IPython.display import display

# Assuming you have multiple DataFrames df1, df2, df3
display(result_df_ridge)
display(result_df_ols)
display(result_df_lasso)

,alpha,degree,mse_train,mse_test,cv_train_mse,cv_test_mse
0,0.00001,1,0.328175,0.513264,0.365861,0.499952
1,0.00001,2,0.327632,0.520977,0.341628,0.528466
2,0.00001,3,0.309951,0.508549,0.304086,0.524125
3,0.00001,4,0.288542,0.425578,0.298241,0.439105
4,0.00001,5,0.287410,0.438121,0.260033,0.494228
...,...,...,...,...,...,...
111,0.00050,25,0.269940,0.458354,0.250401,0.477980
112,0.00050,26,0.269365,0.461027,0.260270,0.464914
113,0.00050,27,0.268820,0.463853,0.250405,0.518133
114,0.00050,28,0.268358,0.466673,0.275493,1.110621


,alpha,degree,mse_train,mse_test,cv_train_mse,cv_test_mse
0,0,1,0.328175,0.513264,0.348661,0.519980
1,0,2,0.327632,0.520977,0.303096,0.525186
2,0,3,0.309951,0.508550,0.278610,0.501519
3,0,4,0.288542,0.425523,0.276690,0.458344
4,0,5,0.287408,0.438487,0.283220,0.460583
5,0,6,0.286801,0.432382,0.272117,0.446846
6,0,7,0.286701,0.433903,0.271964,0.440588
7,0,8,0.278134,0.451145,0.263721,0.478476
8,0,9,0.273189,0.467748,0.247578,0.481368
9,0,10,0.262835,0.446077,0.259802,0.474242


,alpha,degree,mse_train,mse_test,cv_train_mse,cv_test_mse
0,0.00001,1,0.328175,0.513265,0.325079,0.501405
1,0.00001,2,0.328079,0.514023,0.344696,0.516691
2,0.00001,3,0.328043,0.512674,0.310186,0.548769
3,0.00001,4,0.327969,0.511232,0.319316,0.516180
4,0.00001,5,0.327913,0.510325,0.337751,0.525406
...,...,...,...,...,...,...
111,0.00050,25,0.327597,0.512335,0.279485,0.538338
112,0.00050,26,0.327564,0.512686,0.315920,0.558755
113,0.00050,27,0.327530,0.513054,0.313856,0.526258
114,0.00050,28,0.327496,0.513432,0.312278,0.530310
